# Creating a Simple Agent with Tracing

In [1]:
import dotenv
import os

from openai import OpenAI

dotenv.load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    print(
        """Error: OPENAI_API_KEY environment variable not set. Please copy the .env.template file as .env and fill it in.
    
    You can execute these commands in the terminal to get started:
    cp .env.template .env
    code .env
    """
    )

# Test OpenAI Access
print(
    OpenAI()
    .responses.create(
        model=os.environ["OPENAI_DEFAULT_MODEL"], input="Say: We are up and running!"
    )
    .output_text
)

We are up and running!


In [2]:
from agents import Agent, Runner, trace
from openai.types.responses import ResponseTextDeltaEvent

Create a simple Nutrition Assistant Agent

In [3]:
nutrition_agent = Agent(
    name = "Nutrition Assistant", 
    instructions = """
    You are a helpful assistant giving out nutrition advice.
    You give concise answers.
"""
)

Let's execute the Agent:

In [4]:
with trace("Simple Nutrition Agent"):
    result = await Runner.run(nutrition_agent, "How healthy are bananas?")

print(result)

RunResult:
- Last agent: Agent(name="Nutrition Assistant", ...)
- Final output (str):
    Bananas are generally very healthy as part of a balanced diet.
    
    Key benefits:
    - Good source of potassium (supports blood pressure and heart function)
    - Dietary fiber (helps digestion and satiety)
    - Vitamin C, vitamin B6, and small amounts of other nutrients
    - Moderate calories (~105 per medium banana)
    
    Things to note:
    - They contain natural sugars; portion if managing blood sugar or calories.
    - People with kidney disease may need to limit potassium; consult a clinician.
    - Very ripe bananas have more sugar but are still fine in moderation.
    
    Tip: pair with protein or healthy fat (e.g., peanut butter, yogurt) to improve satiety.
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


Streaming the answer to the screen, token by token

In [5]:
response_stream = Runner.run_streamed(nutrition_agent, "How healthy are bananas?")

async for event in response_stream.stream_events():
    if event.type == "raw_response_event" and isinstance(
        event.data, ResponseTextDeltaEvent
    ):
        print(event.data.delta, end="", flush=True)

Bananas are generally healthy. They’re:

- Rich in potassium, vitamin B6, vitamin C, and dietary fiber.
- Low in fat and good for quick energy (mostly from natural sugars and starch).
- Helpful for digestion and heart health, when eaten in moderate amounts.

Things to know:
- If you’re watching sugar or carbs (diabetes, certain diets), portion size matters. A medium banana has ~27 g carbs.
- People with kidney disease on potassium-restricted diets should limit them.
- Riper bananas have more sugar, but similar nutrients.

One medium banana is a solid daily option as part of a balanced diet.

_Good Job!_